In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from statistics import mean
from datetime import timedelta

import math

import warnings  
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Data/consolidatedDataForPackage.csv")
data['Display Time'] = pd.to_datetime(data['Display Time'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])
df = data[data['subjectId']=='1636-69-032']
df = df.reset_index(drop=True)

In [6]:
def fullDay(data):
        
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['Display Time'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

In [13]:
def adrr(x, units):    
    if (units == 'mg'):
        fBG = 1.509*((np.log(x['GlucoseValue']))**1.084  - 5.381)
    elif (units=='mmol'):
        fBG = 1.509*((np.log(18*x['GlucoseValue']) )**1.084  - 5.381)
    else:
        return print('units should be either mmol or mg')
        
    rBG = 10 * fBG ** 2 # called BG risk function
    s = np.sign(fBG)
    s_left = np.abs(s.where(s == -1, 0))
    rlBG = rBG * s_left # called BG risk function left branch

    s_right = s.where(s == 1, 0)
    rhBG = rBG * s_right # called BG risk function right branch

    ADRR = max(rlBG) + max(rhBG) # !!amend the code to output average across days !!!!!
    
    return ADRR
          
#     return pd.DataFrame({'ADRR':[ADRR]}) 


In [14]:
dates = []
for i in range(len(df.index)):
    dates.append(df['Display Time'][i].date())
df['Date'] = dates
df

Display Time  GlucoseValue    subjectId        Date
0    2016-01-13 12:58:17         122.0  1636-69-032  2016-01-13
1    2016-01-13 13:03:17         123.0  1636-69-032  2016-01-13
2    2016-01-13 13:08:17         124.0  1636-69-032  2016-01-13
3    2016-01-13 13:13:17         128.0  1636-69-032  2016-01-13
4    2016-01-13 13:18:17         133.0  1636-69-032  2016-01-13
...                  ...           ...          ...         ...
1778 2016-01-19 17:12:49         101.0  1636-69-032  2016-01-19
1779 2016-01-19 17:17:49          98.0  1636-69-032  2016-01-19
1780 2016-01-19 17:22:49         101.0  1636-69-032  2016-01-19
1781 2016-01-19 17:27:49         106.0  1636-69-032  2016-01-19
1782 2016-01-19 17:32:49         107.0  1636-69-032  2016-01-19

[1783 rows x 4 columns]

In [16]:
adrr_daily = []
for Date, xx in df.groupby('Date'):
    xx = xx.reset_index(drop=True)
    z = adrr(xx,'mg')
    adrr_daily.append(z)

mean(adrr_daily)
    

7.390817595280894